In [ ]:
cd alphafold3
#git clone https://github.com/google-deepmind/alphafold3.git
#git stash
#git pull
#git stash pop
SHORT_HASH=`git rev-parse --short HEAD`
echo $SHORT_HASH
git diff
#cd -

44e1fd5
diff --git a/docker/Dockerfile b/docker/Dockerfile
index 2115d55..60cc73a 100644
--- a/docker/Dockerfile
+++ b/docker/Dockerfile
@@ -51,7 +51,11 @@ RUN build_data
 # when running AlphaFold 3:
 ENV XLA_FLAGS="--xla_gpu_enable_triton_gemm=false"
 # Memory settings used for folding up to 5,120 tokens on A100 80 GB.
-ENV XLA_PYTHON_CLIENT_PREALLOCATE=true
-ENV XLA_CLIENT_MEM_FRACTION=0.95
+#ENV XLA_PYTHON_CLIENT_PREALLOCATE=true
+#ENV XLA_CLIENT_MEM_FRACTION=0.95
+# https://github.com/google-deepmind/alphafold3/blob/main/docs/performance.md#unified-memory
+ENV XLA_PYTHON_CLIENT_PREALLOCATE=false
+ENV TF_FORCE_UNIFIED_MEMORY=true
+ENV XLA_CLIENT_MEM_FRACTION=3.2
 
 CMD ["python3", "run_alphafold.py"]
diff --git a/src/alphafold3/model/model_config.py b/src/alphafold3/model/model_config.py
index 65f97b2..0a2d1f2 100644
--- a/src/alphafold3/model/model_config.py
+++ b/src/alphafold3/model/model_config.py
@@ -26,7 +26,9 @@ class GlobalConfig(base_config.BaseConfig):
   pair_attention_ch

In [6]:
cd alphafold3
docker buildx build --platform=linux/amd64 --progress plain -t alphafold3-$SHORT_HASH-lowspec -f docker/Dockerfile .
cd -

#0 building with "desktop-linux" instance using docker driver

#1 [internal] load build definition from Dockerfile
#1 transferring dockerfile: 2.93kB done
#1 DONE 0.0s

#2 [internal] load metadata for docker.io/nvidia/cuda:12.6.0-base-ubuntu22.04
#2 DONE 1.5s

#3 [internal] load .dockerignore
#3 transferring context: 2B done
#3 DONE 0.0s

#4 [internal] load build context
#4 transferring context: 19.68kB 0.0s done
#4 DONE 0.0s

#5 [ 1/10] FROM docker.io/nvidia/cuda:12.6.0-base-ubuntu22.04@sha256:5dca947f477ec8ea91624447bdad7cc5f6a0dfe038ef53642c884bf5416478a6
#5 resolve docker.io/nvidia/cuda:12.6.0-base-ubuntu22.04@sha256:5dca947f477ec8ea91624447bdad7cc5f6a0dfe038ef53642c884bf5416478a6 done
#5 sha256:5dca947f477ec8ea91624447bdad7cc5f6a0dfe038ef53642c884bf5416478a6 743B / 743B done
#5 sha256:5a05e85d68056b2b095db5fae15d633335d6447f05fd82e3d9450f4977b6fd3e 1.37kB / 1.37kB done
#5 sha256:52696f628dca15881072daff0433dd4206e0d934a4242f945830c242477fda5e 9.69kB / 9.69kB done
#5 sha256:3713021

In [8]:
docker image ls

REPOSITORY                   TAG       IMAGE ID       CREATED          SIZE
alphafold3-44e1fd5-lowspec   latest    fcc3fc3375b7   20 seconds ago   9.03GB


In [9]:
#docker image rm 5b031e7d28fc
#docker system prune --all --force --volumes
#mkdir -p images
#docker save de9169bdbfeb -o images/alphafold3.tar
docker save fcc3fc3375b7 -o images/alphafold3-$SHORT_HASH-lowspec.tar

In [11]:
SRC='./images/'
DEST='euler.ethz.ch:~/batch-infer/software/alphafold3/images'
rsync -azv --progress $SRC $DEST

building file list ... 
2 files to consider
./
alphafold3-44e1fd5-lowspec.tar
  9058677760 100%   27.80MB/s    0:05:10 (xfer#1, to-check=0/2)

sent 5310441947 bytes  received 48 bytes  16517704.49 bytes/sec
total size is 9058677760  speedup is 1.71
